In [ ]:
import torch
import torch.nn as nn
import os
import glob
from random import shuffle, randint
import os
import datetime
import logging
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

In [ ]:
from FTDDataset.FTDDataset import FloorTypeDetectionDataset
from models.model_builder import model_builder
from trainer import Trainer
from evaluation import evaluate, load_state_dict
from visualization.visualization import visualize_data_sample_or_batch, visualize_weights_of_dense_layer, visualize_img_as_path_seq
from custom_utils.utils import gen_run_dir, CustomLogger, store_used_config, load_run_path_config
from torch.utils.data import DataLoader

from models.classification_heads import Dense_ClassificationHead
from models.fusion_models import Concatenate_FusionModel, CrossModalTransformer_FusionModel
from models.modality_nets import LeNet2dLike_ModalityNet, VggLike_ModalityNet, LeNet1dLike_ModalityNet, ImagePatchTokenization_ModalityNet
from models.transformers import CrossModalTransformer
from models.tokenEmbeddings import create_patch_sequence_for_image, flatten_patches

In [ ]:
sensors = None
logger = None
run_path = ""

# ####### load config based on run_path #######
train_config_dict = load_run_path_config(run_path)
train_dataset_path = train_config_dict["train_dataset_path"]
test_dataset_path = train_config_dict["test_dataset_path"]
if sensors == None:
    # use sensors from default config if no sensors were provided as function parameter
    sensors = train_config_dict["sensors"]
else:
    # otherwise use sensors from function parameter and log them in config dict
    train_config_dict["sensors"] = sensors

# ####### start of program (only modify code below if you want to change some behavior) #######
run_paths_dict = gen_run_dir(run_path)
# create new logger object if non was provided
# if logger == None:
#   logger = CustomLogger()

# logger.start_logger(run_paths_dict["logs_path"], stream_log=True)

# ####### prepare datasets #######
# load datasets
ds_train = FloorTypeDetectionDataset(
    train_dataset_path, sensors, run_path)
ds_test = FloorTypeDetectionDataset(
    test_dataset_path, sensors, run_path, create_faulty_data=True)  # only test dataset shall contain faulty data

# ####### define model (automatically select uni or multimodal model) #######
# model = model_builder(sensors, ds_train, train_config_dict)

sample_loader = DataLoader(ds_train, 8, shuffle=True)
_, (sample_batch, _) = next(enumerate(sample_loader))

In [ ]:
test_img = sample_batch[sensors[0]][0]


normal_img = np.transpose(test_img, (1, 2, 0))
plt.imshow(normal_img)

zwerg = create_patch_sequence_for_image(test_img, 16)

visualize_img_as_path_seq(zwerg)